In [1]:
import os
import math
import json
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
import pickle
import openai
from collections import Counter
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [7]:
message = "HI, Do u know whether I can bring meat to Taiwan"
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"user", "content": message},
        {"role": "system", "content": "You are a helpful assistant and reply message in Chinese in one sentence."},
        ]
)
print(response.choices[0].message.content)

對不起，您不能將肉類帶到台灣。


In [12]:
try:
    file_upload = openai.files.create(
        file=open("training_data.jsonl", "rb"),
        purpose="fine-tune"
    )
    training_file_id = file_upload.id
    print(f"檔案上傳成功，檔案 ID: {training_file_id}")

except Exception as e:
    print(f"檔案上傳失敗: {e}")
    training_file_id = None # 如果上傳失敗，後續步驟將無法執行

檔案上傳成功，檔案 ID: file-BgfXbHiiA36ku5P631nZTb


In [13]:
if training_file_id:
    try:
        fine_tune_job = openai.fine_tuning.jobs.create(
            training_file=training_file_id,
            model="gpt-3.5-turbo" # 或其他支援微調的模型，例如 gpt-4o-mini
        )
        job_id = fine_tune_job.id
        print(f"微調任務建立成功，任務 ID: {job_id}")
        print(f"任務狀態: {fine_tune_job.status}")
        print("您可以透過任務 ID 監控進度。")

    except Exception as e:
        print(f"微調任務建立失敗: {e}")
        job_id = None

微調任務建立成功，任務 ID: ftjob-E6ie3L5KfcK8FuSs90PoRxNb
任務狀態: validating_files
您可以透過任務 ID 監控進度。


In [20]:
import time
def check_finetune_status(job_id):
    job_status = openai.fine_tuning.jobs.retrieve(job_id)
    print(f"當前任務狀態: {job_status.status}")
    return job_status.status
if job_id:
    while check_finetune_status(job_id) != "failed" and check_finetune_status(job_id) != "succeeded":
        time.sleep(1)
    try:
        # 獲取任務事件列表，查看進度或錯誤
        # 注意：這可能不會立即顯示所有事件，需要等待一段時間
        events = openai.fine_tuning.jobs.list_events(job_id=job_id, limit=50)
        print("最近的任務事件:")
        for event in events.data:
            print(f"- {event.created_at}: {event.message}")

    except Exception as e:
        print(f"監控微調任務失敗: {e}")


當前任務狀態: succeeded
當前任務狀態: succeeded
監控微調任務失敗: Jobs.list_events() got an unexpected keyword argument 'job_id'


In [23]:
fine_tuned_model_name = "ft:gpt-3.5-turbo-0125:testmodel::BYy1McvE" # 請替換為您的實際模型名稱

print(f"正在嘗試調用微調模型: {fine_tuned_model_name}")
message = "May I can bring meat to Taiwan"
response = openai.chat.completions.create(
    model=fine_tuned_model_name,
    messages=[
        {"role":"user", "content": message},
        {"role": "system", "content": "reply message in Chinese in one sentence."},
        ]
)
print(response.choices[0].message.content)

正在嘗試調用微調模型: ft:gpt-3.5-turbo-0125:testmodel::BYy1McvE
你可以帶熟的肉類，不能帶生的肉類
